###  This is a template for the get_date monte carlo function.  

In [64]:
def get_date(start_date, end_date, delta_time) -> datetime:
    
    import random
    
    # initialize date list with start_date and increment by delta_time
    dates = start_date
    date_array = [start_date]
    while dates < end_date:
        date_array.append(dates)
        dates += delta_time

    # randomly select time based on delta_time increment
    return(random.choice(date_array))

In [66]:
from datetime import datetime, date, timedelta

start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(minutes = 30)
spill_date = get_date(start_date, end_date, delta_time)

In [69]:
spill_date

datetime.datetime(2017, 2, 9, 3, 0)

In [67]:
spill_date.month

2

In [68]:
spill_date.hour

3